In [1]:
import os
import re
import pyspark
from pyspark.sql import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import *
conf = pyspark.SparkConf()
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
#conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','4g')

sc = pyspark.SparkContext(conf=conf)
sc.setLogLevel("ERROR")
spark = pyspark.SQLContext.getOrCreate(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/30 20:28:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/opt/conda/envs/bigdata/lib/python3.10/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


## Load data

In [2]:
# Load csv file
weather_data = spark.read\
              .option("inferSchema", "true")\
              .option("header", "true")\
              .csv("weather_data_filtered.csv")

In [3]:
weather_data.printSchema()

root
 |-- utc_timestamp: timestamp (nullable = true)
 |-- AT_temperature: double (nullable = true)
 |-- AT_radiation_direct_horizontal: double (nullable = true)
 |-- AT_radiation_diffuse_horizontal: double (nullable = true)
 |-- BE_temperature: double (nullable = true)
 |-- BE_radiation_direct_horizontal: double (nullable = true)
 |-- BE_radiation_diffuse_horizontal: double (nullable = true)
 |-- BG_temperature: double (nullable = true)
 |-- BG_radiation_direct_horizontal: double (nullable = true)
 |-- BG_radiation_diffuse_horizontal: double (nullable = true)
 |-- CH_temperature: double (nullable = true)
 |-- CH_radiation_direct_horizontal: double (nullable = true)
 |-- CH_radiation_diffuse_horizontal: double (nullable = true)
 |-- CZ_temperature: double (nullable = true)
 |-- CZ_radiation_direct_horizontal: double (nullable = true)
 |-- CZ_radiation_diffuse_horizontal: double (nullable = true)
 |-- DE_temperature: double (nullable = true)
 |-- DE_radiation_direct_horizontal: double (n

In [4]:
weather_data_DE = weather_data.select("utc_timestamp","DE_temperature", "DE_radiation_direct_horizontal", "DE_radiation_diffuse_horizontal")

In [5]:
weather_data_DE.show(10)

+-------------------+--------------+------------------------------+-------------------------------+
|      utc_timestamp|DE_temperature|DE_radiation_direct_horizontal|DE_radiation_diffuse_horizontal|
+-------------------+--------------+------------------------------+-------------------------------+
|2014-12-31 19:00:00|        -0.981|                           0.0|                            0.0|
|2014-12-31 20:00:00|        -1.035|                           0.0|                            0.0|
|2014-12-31 21:00:00|        -1.109|                           0.0|                            0.0|
|2014-12-31 22:00:00|        -1.166|                           0.0|                            0.0|
|2014-12-31 23:00:00|        -1.226|                           0.0|                            0.0|
|2015-01-01 00:00:00|        -1.305|                           0.0|                            0.0|
|2015-01-01 01:00:00|        -1.478|                           0.0|                            0.0|


In [6]:
weather_data_DE.count()

43824

In [7]:
weather_data_DE.head(1)

[Row(utc_timestamp=datetime.datetime(2014, 12, 31, 19, 0), DE_temperature=-0.981, DE_radiation_direct_horizontal=0.0, DE_radiation_diffuse_horizontal=0.0)]

In [8]:
weather_data_DE.tail(1)

[Row(utc_timestamp=datetime.datetime(2019, 12, 31, 18, 0), DE_temperature=-0.044, DE_radiation_direct_horizontal=0.0, DE_radiation_diffuse_horizontal=0.0)]

In [9]:
# datetime.datetime(2014, 12, 31, 19, 0) - datetime.datetime(2019, 12, 31, 18, 0)

In [10]:
# Load csv file
time_series = spark.read\
              .option("inferSchema", "true")\
              .option("header", "true")\
              .csv("time_series_60min_singleindex_DE.csv")

In [11]:
time_series.printSchema()

root
 |-- utc_timestamp: timestamp (nullable = true)
 |-- cet_cest_timestamp: timestamp (nullable = true)
 |-- DE_load_actual_entsoe_transparency: integer (nullable = true)
 |-- DE_load_forecast_entsoe_transparency: integer (nullable = true)
 |-- DE_solar_capacity: integer (nullable = true)
 |-- DE_solar_generation_actual: integer (nullable = true)
 |-- DE_solar_profile: double (nullable = true)
 |-- DE_wind_capacity: integer (nullable = true)
 |-- DE_wind_generation_actual: integer (nullable = true)
 |-- DE_wind_profile: double (nullable = true)
 |-- DE_wind_offshore_capacity: integer (nullable = true)
 |-- DE_wind_offshore_generation_actual: integer (nullable = true)
 |-- DE_wind_offshore_profile: double (nullable = true)
 |-- DE_wind_onshore_capacity: integer (nullable = true)
 |-- DE_wind_onshore_generation_actual: integer (nullable = true)
 |-- DE_wind_onshore_profile: double (nullable = true)



In [12]:
time_series.count()

43824

In [13]:
time_series_DE = time_series.select("utc_timestamp", "DE_solar_generation_actual","DE_wind_generation_actual")

In [14]:
time_series_DE.show(10)

+-------------------+--------------------------+-------------------------+
|      utc_timestamp|DE_solar_generation_actual|DE_wind_generation_actual|
+-------------------+--------------------------+-------------------------+
|2014-12-31 18:00:00|                      NULL|                     NULL|
|2014-12-31 19:00:00|                      NULL|                     8852|
|2014-12-31 20:00:00|                      NULL|                     9054|
|2014-12-31 21:00:00|                      NULL|                     9070|
|2014-12-31 22:00:00|                      NULL|                     9163|
|2014-12-31 23:00:00|                      NULL|                     9231|
|2015-01-01 00:00:00|                      NULL|                     9689|
|2015-01-01 01:00:00|                      NULL|                    10331|
|2015-01-01 02:00:00|                        71|                    10208|
|2015-01-01 03:00:00|                       773|                    10029|
+-------------------+----

In [15]:
time_series_DE.head(1)

[Row(utc_timestamp=datetime.datetime(2014, 12, 31, 18, 0), DE_solar_generation_actual=None, DE_wind_generation_actual=None)]

In [16]:
time_series_DE.tail(1)

[Row(utc_timestamp=datetime.datetime(2019, 12, 31, 17, 0), DE_solar_generation_actual=0, DE_wind_generation_actual=6573)]

## Join datasets

In [17]:
df_DE = weather_data_DE.join(time_series_DE, weather_data_DE.utc_timestamp == time_series_DE.utc_timestamp)

In [18]:
df_DE.count()

43823

In [19]:
df_DE.show(5)

+-------------------+--------------+------------------------------+-------------------------------+-------------------+--------------------------+-------------------------+
|      utc_timestamp|DE_temperature|DE_radiation_direct_horizontal|DE_radiation_diffuse_horizontal|      utc_timestamp|DE_solar_generation_actual|DE_wind_generation_actual|
+-------------------+--------------+------------------------------+-------------------------------+-------------------+--------------------------+-------------------------+
|2014-12-31 19:00:00|        -0.981|                           0.0|                            0.0|2014-12-31 19:00:00|                      NULL|                     8852|
|2014-12-31 20:00:00|        -1.035|                           0.0|                            0.0|2014-12-31 20:00:00|                      NULL|                     9054|
|2014-12-31 21:00:00|        -1.109|                           0.0|                            0.0|2014-12-31 21:00:00|                

In [20]:
df_DE_solar =df_DE.select("DE_temperature", "DE_radiation_direct_horizontal", "DE_radiation_diffuse_horizontal", "DE_solar_generation_actual")

In [21]:
df_DE_solar.show(10)

+--------------+------------------------------+-------------------------------+--------------------------+
|DE_temperature|DE_radiation_direct_horizontal|DE_radiation_diffuse_horizontal|DE_solar_generation_actual|
+--------------+------------------------------+-------------------------------+--------------------------+
|        -0.981|                           0.0|                            0.0|                      NULL|
|        -1.035|                           0.0|                            0.0|                      NULL|
|        -1.109|                           0.0|                            0.0|                      NULL|
|        -1.166|                           0.0|                            0.0|                      NULL|
|        -1.226|                           0.0|                            0.0|                      NULL|
|        -1.305|                           0.0|                            0.0|                      NULL|
|        -1.478|                     

In [22]:
df_DE_solar.summary().show()

+-------+-----------------+------------------------------+-------------------------------+--------------------------+
|summary|   DE_temperature|DE_radiation_direct_horizontal|DE_radiation_diffuse_horizontal|DE_solar_generation_actual|
+-------+-----------------+------------------------------+-------------------------------+--------------------------+
|  count|            43823|                         43823|                          43823|                     43720|
|   mean|9.850944458389428|             82.75276786541987|              67.07522581369929|         4304.664775846295|
| stddev|8.130191850223826|            159.00064157148347|              89.17442660813396|         6603.901406183148|
|    min|          -12.686|                           0.0|                            0.0|                         0|
|    25%|            3.259|                           0.0|                            0.0|                         0|
|    50%|            9.236|                0.42089684974

In [23]:
df_DE_solar = df_DE_solar.na.fill(0)

In [24]:
df_DE_solar.summary().show()

+-------+-----------------+------------------------------+-------------------------------+--------------------------+
|summary|   DE_temperature|DE_radiation_direct_horizontal|DE_radiation_diffuse_horizontal|DE_solar_generation_actual|
+-------+-----------------+------------------------------+-------------------------------+--------------------------+
|  count|            43823|                         43823|                          43823|                     43823|
|   mean|9.850944458389428|             82.75276786541987|              67.07522581369929|         4294.547246879492|
| stddev|8.130191850223826|            159.00064157148347|              89.17442660813396|        6599.4287412459225|
|    min|          -12.686|                           0.0|                            0.0|                         0|
|    25%|            3.259|                           0.0|                            0.0|                         0|
|    50%|            9.236|                0.42089684974

In [25]:
df_DE_wind =df_DE.select("DE_temperature", "DE_radiation_direct_horizontal", "DE_radiation_diffuse_horizontal", "DE_wind_generation_actual")

In [26]:
df_DE_wind.show(10)

+--------------+------------------------------+-------------------------------+-------------------------+
|DE_temperature|DE_radiation_direct_horizontal|DE_radiation_diffuse_horizontal|DE_wind_generation_actual|
+--------------+------------------------------+-------------------------------+-------------------------+
|        -0.981|                           0.0|                            0.0|                     8852|
|        -1.035|                           0.0|                            0.0|                     9054|
|        -1.109|                           0.0|                            0.0|                     9070|
|        -1.166|                           0.0|                            0.0|                     9163|
|        -1.226|                           0.0|                            0.0|                     9231|
|        -1.305|                           0.0|                            0.0|                     9689|
|        -1.478|                           0.0

In [27]:
df_DE_wind.summary().show()

+-------+-----------------+------------------------------+-------------------------------+-------------------------+
|summary|   DE_temperature|DE_radiation_direct_horizontal|DE_radiation_diffuse_horizontal|DE_wind_generation_actual|
+-------+-----------------+------------------------------+-------------------------------+-------------------------+
|  count|            43823|                         43823|                          43823|                    43749|
|   mean|9.850944458389428|             82.75276786541987|              67.07522581369929|       11157.660037943724|
| stddev|8.130191850223826|            159.00064157148347|              89.17442660813396|        8694.210223442207|
|    min|          -12.686|                           0.0|                            0.0|                      135|
|    25%|            3.259|                           0.0|                            0.0|                     4422|
|    50%|            9.236|                0.420896849743|      

In [28]:
df_DE_wind = df_DE_wind.na.fill(0)

In [29]:
df_DE_wind.summary().show()

+-------+-----------------+------------------------------+-------------------------------+-------------------------+
|summary|   DE_temperature|DE_radiation_direct_horizontal|DE_radiation_diffuse_horizontal|DE_wind_generation_actual|
+-------+-----------------+------------------------------+-------------------------------+-------------------------+
|  count|            43823|                         43823|                          43823|                    43823|
|   mean|9.850944458389428|             82.75276786541987|              67.07522581369929|       11138.819090431965|
| stddev|8.130191850223826|            159.00064157148347|              89.17442660813396|        8698.937767472748|
|    min|          -12.686|                           0.0|                            0.0|                        0|
|    25%|            3.259|                           0.0|                            0.0|                     4405|
|    50%|            9.236|                0.420896849743|      

## Prediction model

In [30]:
df_DE_solar.show(5)

+--------------+------------------------------+-------------------------------+--------------------------+
|DE_temperature|DE_radiation_direct_horizontal|DE_radiation_diffuse_horizontal|DE_solar_generation_actual|
+--------------+------------------------------+-------------------------------+--------------------------+
|        -0.981|                           0.0|                            0.0|                         0|
|        -1.035|                           0.0|                            0.0|                         0|
|        -1.109|                           0.0|                            0.0|                         0|
|        -1.166|                           0.0|                            0.0|                         0|
|        -1.226|                           0.0|                            0.0|                         0|
+--------------+------------------------------+-------------------------------+--------------------------+
only showing top 5 rows



In [31]:
df_DE_wind.show(5)

+--------------+------------------------------+-------------------------------+-------------------------+
|DE_temperature|DE_radiation_direct_horizontal|DE_radiation_diffuse_horizontal|DE_wind_generation_actual|
+--------------+------------------------------+-------------------------------+-------------------------+
|        -0.981|                           0.0|                            0.0|                     8852|
|        -1.035|                           0.0|                            0.0|                     9054|
|        -1.109|                           0.0|                            0.0|                     9070|
|        -1.166|                           0.0|                            0.0|                     9163|
|        -1.226|                           0.0|                            0.0|                     9231|
+--------------+------------------------------+-------------------------------+-------------------------+
only showing top 5 rows

